<a href="https://colab.research.google.com/github/JamesPeralta/Machine-Learning-Algorithms/blob/master/Generative%20Models/DeepDream/Implementing_DeepDream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing DeepDream in Keras

## Start from a convnet pretrained on ImageNet
### VGG16, VGG19, Xception, ResNet50, etc. can implement DeepDream, but your convnet of choice will naturally affect your visualizations, because different convnet architectures result in different learned features.

In [0]:
from keras.applications import inception_v3
from keras import backend as K

In [0]:
from google.colab import drive
import os

In [21]:
drive.mount('/content/drive')
picture_location = '/content/drive/My Drive/Datasets/DeepDream/Brian'

os.chdir(picture_location)
os.listdir()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['Brian.png']

In [0]:
K.set_learning_phase(0)

# For our example we will be using the Inception V3 network
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

## Compute the loss
### The quantity you’ll seek to maximize during the gradient-ascent process. For filter visulization, I maximized the value of a specific filter in a specific layer. Here, ill simultaneously maximize the activation of all filters in a number of layers. 
### Specifically, you’ll maximize a weighted sum of the L2 norm of the activations of a set of high-level layers. The exact set of layers you choose (as well as their contribution to the final loss) has a major influence on the visuals you’ll be able to produce, so you want to make these parameters easily configurable.

In [23]:
# Setting up the DeepDream configuration
layer_contributions = {
    'mixed7': 0.2,    # Dictionary mapping layer names to a coefficient quantifying
    'mixed8': 3.,     # how much the layer’s activation contributes to the loss  
    'mixed9': 2.,     # you’ll seek to maximize. Note that the layer names are
    'mixed10': 1.5,    # hardcoded in the built-in Inception V3 application. You can
}                     # list all layer names using model.summary().

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, None, None, 3 96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_95 (Activation)      (None, None, None, 3 0           batch_normalization_95[0][0]     
__________________________________________________________________________________________________
conv2d_96 

In [10]:
layer_dict = dict([(layer.name, layer) for layer in model.layers]) # Creates a dictionary that maps layer names to layer instances

loss = K.variable(0.) # Define the loss by adding layer contribution to this scalar variable

for layer_name in layer_contributions:
  coeff = layer_contributions[layer_name]
  activation = layer_dict[layer_name].output # Retrieves the layer's output array
  scaling = K.prod(K.cast(K.shape(activation), 'float32'))
  loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

In [0]:
dream = model.input # This tensor holds the dream

grads = K.gradients(loss, dream)[0] # Computes the gradients (important trick)

grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

outputs = [loss, grads]                              # Sets up a Keras function to retrieve the value of 
fetch_loss_and_grads = K.function([dream], outputs)  # the loss and gradients, given an input image

def eval_loss_and_grads(x):
  outs = fetch_loss_and_grads([x])
  loss_value = outs[0]
  grad_values = outs[1]
  return loss_value, grad_values

# This function runs gradient ascent for a number of iterations.
def gradient_ascent(x, iterations, step, max_loss=None): 
  for i in range(iterations):
     loss_value, grad_values = eval_loss_and_grads(x)
     if max_loss is not None and loss_value > max_loss:
         break
     print('...Loss value at', i, ':', loss_value)
     x += step * grad_values
  return x

## Finally: the actual DeepDream algorithm
### First, define a list of scales (also called octaves) at which to process the images. Each successive scale is larger than the previous one by a factor of 1.4 (it’s 40% larger): you start by processing a small image and then increasingly scale it up
### For each successive scale, from the smallest to the largest, you run gradient ascent to maximize the loss you previously defined, at that scale. After each gradient ascent run, you upscale the resulting image by 40%.
### To avoid losing a lot of image detail after each successive scale-up (resulting in increasingly blurry or pixelated images), you can use a simple trick: after each scale- up, you’ll reinject the lost details back into the image,


In [0]:
import numpy as np

In [24]:
step = 0.01 # Gradient ascent step size
num_octave = 3 # Number of otaves (# of upscales)
octave_scale = 1.4 # Size of upscale (in this case 1.4 or 40%)
iterations = 20 # Run gradient ascent for 20 iterations

max_loss = 10.  # If the loss grows larger than 10,
                # you’ll interrupt the gradient-ascent process to avoid ugly artifacts.

base_image_path = 'Brian.png'

img = preprocess_image(base_image_path) # Loads the base image into a Numpy array

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
# Prepares a list of shape tuples defining the different scales at which to run gradient ascent
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i))
        for dim in original_shape])
    successive_shapes.append(shape)
    
successive_shapes = successive_shapes[::-1] # Reverses the list of shapes so they’re in increasing order

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0]) # Resizes the Numpy array of the image to the smallest scale

for shape in successive_shapes:
  print('Processing image shape', shape)
  img = resize_img(img, shape) # Scales up the image shape 
  img = gradient_ascent(img, # Runs gradient ascent, altering the dream
                        iterations=iterations,
                        step=step,
                        max_loss=max_loss)
  
  upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape) 
  same_size_original = resize_img(original_img, shape) # Computes the high-quality version of the original image at this size
  lost_detail = same_size_original - upscaled_shrunk_original_img # The difference between the two is the detail that was lost when scaling up.
  
  img += lost_detail # Reinjects lost detail into the dream
  shrunk_original_img = resize_img(original_img, shape)
  save_img(img, fname='dream_at_scale_' + str(shape) + '.png')

save_img(img, fname='final_dream.png')

/usr/local/lib/python3.6/dist-packages/scipy/ndimage/interpolation.py:583: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


Processing image shape (242, 229)
...Loss value at 0 : 1.6487424
...Loss value at 1 : 1.9464303
...Loss value at 2 : 2.5260575
...Loss value at 3 : 3.2020082
...Loss value at 4 : 3.809022
...Loss value at 5 : 4.4054766
...Loss value at 6 : 5.030923
...Loss value at 7 : 5.662548
...Loss value at 8 : 6.221211
...Loss value at 9 : 6.763938
...Loss value at 10 : 7.3772407
...Loss value at 11 : 7.875053
...Loss value at 12 : 8.3797865
...Loss value at 13 : 8.85789
...Loss value at 14 : 9.308052
...Loss value at 15 : 9.775698


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  # This is added back by InteractiveShellApp.init_path()


Processing image shape (340, 321)
...Loss value at 0 : 3.0728703
...Loss value at 1 : 4.290121
...Loss value at 2 : 5.369831
...Loss value at 3 : 6.271934
...Loss value at 4 : 7.098414
...Loss value at 5 : 7.8373733
...Loss value at 6 : 8.530694
...Loss value at 7 : 9.15915
...Loss value at 8 : 9.773747
Processing image shape (476, 450)
...Loss value at 0 : 3.1943204
...Loss value at 1 : 4.3312073
...Loss value at 2 : 5.356767
...Loss value at 3 : 6.286502
...Loss value at 4 : 7.162728
...Loss value at 5 : 7.941783
...Loss value at 6 : 8.676771
...Loss value at 7 : 9.439405


## Define some helper functions for working with image


In [0]:
import scipy
from keras.preprocessing import image

In [0]:
def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)
  
def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname, pil_img)
    
def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img
  
def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x